In [18]:
from driver import FINNAccelDriver
import numpy as np
from pynq.ps import Clocks
import time

In [19]:
#constants
BATCH_SIZE = 200
XCLBIN_PATH = "a.xclbin"
PLATFORM = "alveo"
INPUT_FILE_PATH = "input.npy"

In [20]:
#Download the CIFAR 10 dataset 
!wget https://raw.githubusercontent.com/modestyachts/CIFAR-10.1/master/datasets/cifar10.1_v4_data.npy
full_cifar = np.load('cifar10.1_v4_data.npy')
partial_cifar = full_cifar[:BATCH_SIZE]
np.save(INPUT_FILE_PATH, partial_cifar)
print(partial_cifar.shape)

--2020-10-27 22:19:55--  https://raw.githubusercontent.com/modestyachts/CIFAR-10.1/master/datasets/cifar10.1_v4_data.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6208640 (5.9M) [application/octet-stream]
Saving to: ‘cifar10.1_v4_data.npy.4’

100%[======================================>] 6,208,640   --.-K/s   in 0.1s    

2020-10-27 22:19:55 (52.1 MB/s) - ‘cifar10.1_v4_data.npy.4’ saved [6208640/6208640]

(200, 32, 32, 3)


In [21]:
finnDriver = FINNAccelDriver(BATCH_SIZE, XCLBIN_PATH, PLATFORM)

ibuf_normal = np.load(INPUT_FILE_PATH)
ibuf_folded = finnDriver.fold_input(ibuf_normal)
ibuf_packed = finnDriver.pack_input(ibuf_folded)
finnDriver.copy_input_data_to_device(ibuf_packed)

In [22]:
start = time.time()
finnDriver.execute()
obuf_packed = np.empty_like(finnDriver.obuf_packed_device)
finnDriver.copy_output_data_from_device(obuf_packed)
obuf_folded = finnDriver.unpack_output(obuf_packed)
obuf_normal = finnDriver.unfold_output(obuf_folded)
end = time.time()
print(obuf_normal.shape)

(200, 1)


In [23]:
runtime = end - start
res = {}
res["runtime[ms]"] = runtime*1000
res["throughput[images/s]"] = BATCH_SIZE / runtime
res["DRAM_in_bandwidth[Mb/s]"] = np.prod(finnDriver.ishape_packed)*0.000001 / runtime
res["DRAM_out_bandwidth[Mb/s]"] = np.prod(finnDriver.oshape_packed)*0.000001 / runtime
if PLATFORM != "alveo":
    res["fclk[mhz]"] = Clocks.fclk0_mhz
else:
    res["fclk[mhz]"] = finnDriver.fclk_mhz
res["N"] = BATCH_SIZE
res

{'runtime[ms]': 70.63817977905273,
 'throughput[images/s]': 2831.3300346296382,
 'DRAM_in_bandwidth[Mb/s]': 8.697845866382249,
 'DRAM_out_bandwidth[Mb/s]': 0.002831330034629638,
 'fclk[mhz]': 100.0,
 'N': 200}